<a href="https://colab.research.google.com/github/kmeng01/rome/blob/main/notebooks/causal_trace.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" align="left"/></a>&nbsp;or in a local notebook.

In [1]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
cd /content && rm -rf /content/rome
git clone https://github.com/kmeng01/rome rome > install.log 2>&1
pip install -r /content/rome/scripts/colab_reqs/rome.txt >> install.log 2>&1
pip install --upgrade google-cloud-storage >> install.log 2>&1

In [2]:
IS_COLAB = False
try:
    import google.colab, torch, os

    IS_COLAB = True
    os.chdir("/content/rome")
    if not torch.cuda.is_available():
        raise Exception("Change runtime type to include a GPU.")
except ModuleNotFoundError as _:
    pass

## Causal Tracing

A demonstration of the double-intervention causal tracing method.

The strategy used by causal tracing is to understand important
states within a transfomer by doing two interventions simultaneously:

1. Corrupt a subset of the input.  In our paper, we corrupt the subject tokens
   to frustrate the ability of the transformer to accurately complete factual
   prompts about the subject.
2. Restore a subset of the internal hidden states.  In our paper, we scan
   hidden states at all layers and all tokens, searching for individual states
   that carry the necessary information for the transformer to recover its
   capability to complete the factual prompt.

The traces of decisive states can be shown on a heatmap.  This notebook
demonstrates the code for conducting causal traces and creating these heatmaps.

The `experiments.causal_trace` module contains a set of functions for running causal traces.

In this notebook, we reproduce, demonstrate and discuss the interesting functions.

We begin by importing several utility functions that deal with tokens and transformer models.

In [4]:
import os, re, json
import torch, numpy
from collections import defaultdict
from util import nethook
from util.globals import DATA_DIR
from experiments.causal_trace import (
    ModelAndTokenizer,
    layername,
    guess_subject,
    plot_trace_heatmap,
)
from experiments.causal_trace import (
    make_inputs,
    decode_tokens,
    find_token_range,
    predict_token,
    predict_from_input,
    collect_embedding_std,
)
from dsets import KnownsDataset

torch.set_grad_enabled(False)

/content/rome/experiments/causal_trace.py:35: SyntaxWarning: invalid escape sequence '\d'
  elif re.match("^[uts][\d\.]+", code):


torch.autograd.grad_mode.set_grad_enabled(mode=False)

Now we load a model and tokenizer, and show that it can complete a couple factual statements correctly.

In [5]:
model_name = "gpt2-xl"  # or "EleutherAI/gpt-j-6B" or "EleutherAI/gpt-neox-20b"
mt = ModelAndTokenizer(
    model_name,
    low_cpu_mem_usage=IS_COLAB,
    torch_dtype=(torch.float16 if "20b" in model_name else None),
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [14]:
# --- CÉLULA DE INVESTIGAÇÃO ---

# Vamos pegar o fato problemático (o número 9, que é o índice 8 na lista)
fact_problematico = dados_controle[8]
print(f"Fato problemático: {fact_problematico}\n")

# Vamos recriar exatamente o que o seu loop faz
prompt_template = fact_problematico["requested_rewrite"]["prompt"]
subject_text = fact_problematico["requested_rewrite"]["subject"]
final_prompt = prompt_template.format(subject_text)

print(f"Prompt final que será enviado ao modelo: '{final_prompt}'")
print(f"Sujeito que estamos procurando: '{subject_text}'\n")

# Agora, vamos fazer o que a função find_token_range faz
# 1. Tokenizar o prompt
inp = make_inputs(mt.tokenizer, [final_prompt])
token_array = inp["input_ids"][0]
print(f"Array de tokens (IDs): {token_array}\n")

# 2. Decodificar de volta para texto, token por token
toks = decode_tokens(mt.tokenizer, token_array)
print(f"Tokens decodificados (como a função os vê): {toks}\n")

# 3. Juntar tudo em uma única string
whole_string = "".join(toks)
print(f"String reconstruída a partir dos tokens: '{whole_string}'\n")

# 4. Tentar encontrar a substring
try:
    char_loc = whole_string.index(subject_text)
    print(f"SUCESSO! A substring '{subject_text}' foi encontrada na posição {char_loc}.")
except ValueError:
    print(f"FALHA! A substring '{subject_text}' NÃO foi encontrada na string reconstruída.")

Fato problemático: {'case_id': 8, 'requested_rewrite': {'prompt': 'A capital da {} é', 'subject': 'França', 'target_true': {'str': ' Paris'}}, 'paraphrase_prompts': [], 'neighborhood_prompts': [], 'attribute_prompts': [], 'generation_prompts': []}

Prompt final que será enviado ao modelo: 'A capital da França é'
Sujeito que estamos procurando: 'França'

Array de tokens (IDs): tensor([   32,  3139, 12379, 40178,    64, 38251], device='cuda:0')

Tokens decodificados (como a função os vê): ['A', ' capital', ' da', ' Franç', 'a', ' é']

String reconstruída a partir dos tokens: 'A capital da França é'

SUCESSO! A substring 'França' foi encontrada na posição 13.


In [20]:
predict_token(
    mt,
    ["Megan Rapinoe plays the sport of", "The Space Needle is in the city of"],
    return_p=True,
)

([' soccer', ' Seattle'], tensor([0.7675, 0.9552], device='cuda:0'))

To obfuscate the subject during Causal Tracing, we use noise sampled from a zero-centered spherical Gaussian, whose stddev is 3 times the $\sigma$ stddev the model's embeddings. Let's compute that value.

In [21]:
knowns = KnownsDataset(DATA_DIR)  # Dataset of known facts
noise_level = 3 * collect_embedding_std(mt, [k["subject"] for k in knowns])
print(f"Using noise level {noise_level}")

data/known_1000.json does not exist. Downloading from https://rome.baulab.info/data/dsets/known_1000.json


100%|██████████| 335k/335k [00:00<00:00, 529kB/s]


Loaded dataset with 1209 elements
Using noise level 0.13462981581687927


## Tracing a single location

The core intervention in causal tracing is captured in this function:

`trace_with_patch` a single causal trace.

It enables running a batch of inferences with two interventions.

  1. Random noise can be added to corrupt the inputs of some of the batch.
  2. At any point, clean non-noised state can be copied over from an
     uncorrupted batch member to other batch members.
  
The convention used by this function is that the zeroth element of the
batch is the uncorrupted run, and the subsequent elements of the batch
are the corrupted runs.  The argument tokens_to_mix specifies an
be corrupted by adding Gaussian noise to the embedding for the batch
inputs other than the first element in the batch.  Alternately,
subsequent runs could be corrupted by simply providing different
input tokens via the passed input batch.

To ensure that corrupted behavior is representative, in practice, we
will actually run several (ten) corrupted runs in the same batch,
each with its own sample of noise.

Then when running, a specified set of hidden states will be uncorrupted
by restoring their values to the same vector that they had in the
zeroth uncorrupted run.  This set of hidden states is listed in
states_to_patch, by listing [(token_index, layername), ...] pairs.
To trace the effect of just a single state, this can be just a single
token/layer pair.  To trace the effect of restoring a set of states,
any number of token indices and layers can be listed.

Note that this function is also in experiments.causal_trace; the code
is shown here to show the logic.

In [7]:
def trace_with_patch(
    model,  # The model
    inp,  # A set of inputs
    states_to_patch,  # A list of (token index, layername) triples to restore
    answers_t,  # Answer probabilities to collect
    tokens_to_mix,  # Range of tokens to corrupt (begin, end)
    noise=0.1,  # Level of noise to add
    trace_layers=None,  # List of traced outputs to return
):
    prng = numpy.random.RandomState(1)  # For reproducibility, use pseudorandom noise
    patch_spec = defaultdict(list)
    for t, l in states_to_patch:
        patch_spec[l].append(t)
    embed_layername = layername(model, 0, "embed")

    def untuple(x):
        return x[0] if isinstance(x, tuple) else x

    # Define the model-patching rule.
    def patch_rep(x, layer):
        if layer == embed_layername:
            # If requested, we corrupt a range of token embeddings on batch items x[1:]
            if tokens_to_mix is not None:
                b, e = tokens_to_mix
                x[1:, b:e] += noise * torch.from_numpy(
                    prng.randn(x.shape[0] - 1, e - b, x.shape[2])
                ).to(x.device)
            return x
        if layer not in patch_spec:
            return x
        # If this layer is in the patch_spec, restore the uncorrupted hidden state
        # for selected tokens.
        h = untuple(x)
        for t in patch_spec[layer]:
            h[1:, t] = h[0, t]
        return x

    # With the patching rules defined, run the patched model in inference.
    additional_layers = [] if trace_layers is None else trace_layers
    with torch.no_grad(), nethook.TraceDict(
        model,
        [embed_layername] + list(patch_spec.keys()) + additional_layers,
        edit_output=patch_rep,
    ) as td:
        outputs_exp = model(**inp)

    # We report softmax probabilities for the answers_t token predictions of interest.
    probs = torch.softmax(outputs_exp.logits[1:, -1, :], dim=1).mean(dim=0)[answers_t]

    # If tracing all layers, collect all activations together to return.
    if trace_layers is not None:
        all_traced = torch.stack(
            [untuple(td[layer].output).detach().cpu() for layer in trace_layers], dim=2
        )
        return probs, all_traced

    return probs

## Scanning all locations

A causal flow heatmap is created by repeating `trace_with_patch` at every individual hidden state, and measuring the impact of restoring state at each location.

The `calculate_hidden_flow` function does this loop.  It handles both the case of restoring a single hidden state, and also restoring MLP or attention states.  Because MLP and attention make small residual contributions, to observe a causal effect in those cases, we need to restore several layers of contributions at once, which is done by `trace_important_window`.

In [8]:
def calculate_hidden_flow(
    mt, prompt, subject, samples=10, noise=0.1, window=10, kind=None
):
    """
    Runs causal tracing over every token/layer combination in the network
    and returns a dictionary numerically summarizing the results.
    """
    inp = make_inputs(mt.tokenizer, [prompt] * (samples + 1))
    with torch.no_grad():
        answer_t, base_score = [d[0] for d in predict_from_input(mt.model, inp)]
    [answer] = decode_tokens(mt.tokenizer, [answer_t])
    e_range = find_token_range(mt.tokenizer, inp["input_ids"][0], subject)
    low_score = trace_with_patch(
        mt.model, inp, [], answer_t, e_range, noise=noise
    ).item()
    if not kind:
        differences = trace_important_states(
            mt.model, mt.num_layers, inp, e_range, answer_t, noise=noise
        )
    else:
        differences = trace_important_window(
            mt.model,
            mt.num_layers,
            inp,
            e_range,
            answer_t,
            noise=noise,
            window=window,
            kind=kind,
        )
    differences = differences.detach().cpu()
    return dict(
        scores=differences,
        low_score=low_score,
        high_score=base_score,
        input_ids=inp["input_ids"][0],
        input_tokens=decode_tokens(mt.tokenizer, inp["input_ids"][0]),
        subject_range=e_range,
        answer=answer,
        window=window,
        kind=kind or "",
    )


def trace_important_states(model, num_layers, inp, e_range, answer_t, noise=0.1):
    ntoks = inp["input_ids"].shape[1]
    table = []
    for tnum in range(ntoks):
        row = []
        for layer in range(0, num_layers):
            r = trace_with_patch(
                model,
                inp,
                [(tnum, layername(model, layer))],
                answer_t,
                tokens_to_mix=e_range,
                noise=noise,
            )
            row.append(r)
        table.append(torch.stack(row))
    return torch.stack(table)


def trace_important_window(
    model, num_layers, inp, e_range, answer_t, kind, window=10, noise=0.1
):
    ntoks = inp["input_ids"].shape[1]
    table = []
    for tnum in range(ntoks):
        row = []
        for layer in range(0, num_layers):
            layerlist = [
                (tnum, layername(model, L, kind))
                for L in range(
                    max(0, layer - window // 2), min(num_layers, layer - (-window // 2))
                )
            ]
            r = trace_with_patch(
                model, inp, layerlist, answer_t, tokens_to_mix=e_range, noise=noise
            )
            row.append(r)
        table.append(torch.stack(row))
    return torch.stack(table)

## Plotting the results

The `plot_trace_heatmap` function draws the data on a heatmap.  That function is not shown here; it is in `experiments.causal_trace`.


In [9]:
def plot_hidden_flow(
    mt,
    prompt,
    subject=None,
    samples=10,
    noise=0.1,
    window=10,
    kind=None,
    modelname=None,
    savepdf=None,
):
    if subject is None:
        subject = guess_subject(prompt)
    result = calculate_hidden_flow(
        mt, prompt, subject, samples=samples, noise=noise, window=window, kind=kind
    )
    plot_trace_heatmap(result, savepdf, modelname=modelname)


def plot_all_flow(mt, prompt, subject=None, noise=0.1, modelname=None):
    for kind in [None, "mlp", "attn"]:
        plot_hidden_flow(
            mt, prompt, subject, modelname=modelname, noise=noise, kind=kind
        )

In [18]:
import json

# Caminhos para os seus arquivos de dados
caminho_controle = "dsets/controle.json"
caminho_confirmacao = "dsets/confirmacao.json"
caminho_conflito = "dsets/conflito.json"

# Carregar os dados dos arquivos JSON para listas em Python
with open(caminho_controle, 'r', encoding='utf-8') as f:
    dados_controle = json.load(f)

with open(caminho_confirmacao, 'r', encoding='utf-8') as f:
    dados_confirmacao = json.load(f)

with open(caminho_conflito, 'r', encoding='utf-8') as f:
    dados_conflito = json.load(f)

# Verificação para garantir que tudo foi carregado corretamente
print(f"Dados de Controle carregados: {len(dados_controle)} fatos.")
print(f"Dados de Confirmação carregados: {len(dados_confirmacao)} fatos.")
print(f"Dados de Conflito carregados: {len(dados_conflito)} fatos.")

# Mostra o primeiro item do dataset de controle como exemplo
print("\nExemplo do primeiro item do dataset de controle:")
print(dados_controle[0])



Dados de Controle carregados: 25 fatos.
Dados de Confirmação carregados: 25 fatos.
Dados de Conflito carregados: 25 fatos.

Exemplo do primeiro item do dataset de controle:
{'case_id': 0, 'requested_rewrite': {'prompt': 'O autor de "{}" é', 'subject': 'A Divina Comédia', 'target_true': {'str': ' Dante Alighieri'}}, 'paraphrase_prompts': [], 'neighborhood_prompts': [], 'attribute_prompts': [], 'generation_prompts': []}


In [23]:
import os
import numpy
import torch

# --- Criar pastas para os resultados ---
os.makedirs("results/controle", exist_ok=True)
os.makedirs("results/confirmacao", exist_ok=True)
os.makedirs("results/conflito", exist_ok=True)

# ---------------------------------------------------
# --- Análise para a Condição de CONTROLE ---
# ---------------------------------------------------
#print("Iniciando análise para a condição de CONTROLE...")
#for i, fact in enumerate(dados_controle):
    # Monta o prompt final a partir dos dados corrigidos
#    prompt_template = fact["requested_rewrite"]["prompt"]
#    subject_text = fact["requested_rewrite"]["subject"]
#    final_prompt = prompt_template.format(subject_text)

    # Chama a função com os dados corretos
#    result = calculate_hidden_flow(
#        mt,
#        prompt=final_prompt,
#        subject=subject_text,
#        noise=noise_level
#    )

#    numpy_result = {k: v.cpu().numpy() if isinstance(v, torch.Tensor) else v for k, v in result.items()}
#    numpy.savez(f"results/controle/fact_{i}.npz", **numpy_result)

#    print(f"  Fato {i+1}/{len(dados_controle)} ('{subject_text}') concluído.")


# ---------------------------------------------------
# --- Análise para a Condição de CONFIRMAÇÃO ---
# ---------------------------------------------------

#print("\nIniciando análise para a condição de CONFIRMAÇÃO...")

#for i, fact in enumerate(dados_confirmacao):
#     prompt_template = fact["requested_rewrite"]["prompt"]
#     subject_text = fact["requested_rewrite"]["subject"]
#     final_prompt = prompt_template.format(subject_text)

#     result = calculate_hidden_flow(
#         mt,
#         prompt=final_prompt,
#         subject=subject_text,
#         noise=noise_level
#     )

#     numpy_result = {}
#     for key, value in result.items():
#         if isinstance(value, torch.Tensor):
#             numpy_result[key] = value.cpu().numpy()
#         else:
#             numpy_result[key] = value

#     numpy.savez(f"results/confirmacao/fact_{i}.npz", **numpy_result)
#     print(f"  Fato {i+1}/{len(dados_confirmacao)} ('{subject_text}') concluído.")

#print("\n--- Análise da Condição de Confirmação CONCLUÍDA! ---")


# ---------------------------------------------------
# --- Análise para a Condição de CONFLITO ---
# ---------------------------------------------------
# print("\nIniciando análise para a condição de CONFLITO...")
# for i, fact in enumerate(dados_conflito):
#     prompt_template = fact["requested_rewrite"]["prompt"]
#     subject_text = fact["requested_rewrite"]["subject"]
#     final_prompt = prompt_template.format(subject_text)
#
#     result = calculate_hidden_flow(
#         mt,
#         prompt=final_prompt,
#         subject=subject_text,
#         noise=noise_level
#     )
#
#     numpy_result = {}
#     for key, value in result.items():
#         if isinstance(value, torch.Tensor):
#             numpy_result[key] = value.cpu().numpy()
#         else:
#             numpy_result[key] = value
#
#     numpy.savez(f"results/conflito/fact_{i}.npz", **numpy_result)
#     print(f"  Fato {i+1}/{len(dados_conflito)} ('{subject_text}') concluído.")
#
# print("\n--- Análise da Condição de Conflito CONCLUÍDA! ---")

Iniciando análise para a condição de CONTROLE...
  Fato 1/25 ('A Divina Comédia') concluído.
  Fato 2/25 ('O Monte Everest') concluído.
  Fato 3/25 ('Holanda') concluído.
  Fato 4/25 ('Grande Muralha') concluído.
  Fato 5/25 ('Coliseu') concluído.
  Fato 6/25 ('deserto do Saara') concluído.
  Fato 7/25 ('pirâmides de Gizé') concluído.
  Fato 8/25 ('Cristo Redentor') concluído.
  Fato 9/25 ('França') concluído.
  Fato 10/25 ('Sol') concluído.
  Fato 11/25 ('fotossíntese') concluído.
  Fato 12/25 ('baleia-azul') concluído.
  Fato 13/25 ('Albert Einstein') concluído.
  Fato 14/25 ('alumínio') concluído.
  Fato 15/25 ('gravidade') concluído.
  Fato 16/25 ('Lua') concluído.
  Fato 17/25 ('Dom Quixote') concluído.
  Fato 18/25 ('Mona Lisa') concluído.
  Fato 19/25 ('Segunda Guerra Mundial') concluído.
  Fato 20/25 ('lâmpada elétrica') concluído.
  Fato 21/25 ('Muro de Berlim') concluído.
  Fato 22/25 ('Star Wars') concluído.
  Fato 23/25 ('Queen') concluído.
  Fato 24/25 ('Harry Potter') con

The following prompt can be changed to any factual statement to trace.

In [ ]:
#plot_all_flow(mt, "The Space Needle is in the city of", noise=noise_level)

Here we trace a few more factual statements from a file of test cases.

In [ ]:
#for knowledge in knowns[:5]:
#    plot_all_flow(mt, knowledge["prompt"], knowledge["subject"], noise=noise_level)